In [46]:
#Importar librerías y cargar base de datos

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("incident_event_log.csv")

In [47]:
#Reemplazar "?" por NaN
df=df.replace("?",np.nan)

# Total de filas
total_filas = len(df)

# Conteo de vacíos
faltantes = df.isna().sum()

# Porcentaje de vacíos
porcentaje_faltantes = (faltantes / total_filas) * 100

# DataFrame resumen
faltantes_df = pd.DataFrame({
    "Valores_faltantes": faltantes,
    "Porcentaje_faltantes": porcentaje_faltantes.round(2)
})

print(faltantes_df)

# --- Análisis de duplicados ---

# Número total de filas duplicadas
total_duplicados = df.duplicated().sum()
print(f"Total de filas duplicadas: {total_duplicados}")

# Mostrar las filas duplicadas (si existen)
if total_duplicados > 0:
    print("\n=== Filas duplicadas ===")
    display(df[df.duplicated(keep=False)])  # keep=False muestra todas las ocurrencias
else:
    print("\nNo se encontraron filas duplicadas.")

                         Valores_faltantes  Porcentaje_faltantes
number                                   0                  0.00
incident_state                           0                  0.00
active                                   0                  0.00
reassignment_count                       0                  0.00
reopen_count                             0                  0.00
sys_mod_count                            0                  0.00
made_sla                                 0                  0.00
caller_id                               29                  0.02
opened_by                             4835                  3.41
opened_at                                0                  0.00
sys_created_by                       53076                 37.45
sys_created_at                       53076                 37.45
sys_updated_by                           0                  0.00
sys_updated_at                           0                  0.00
contact_type             

In [48]:
# --- Calcular porcentaje de nulos por columna ---
porc_nulos = (df.isna().sum() / len(df)) * 100

print("=== Porcentaje de valores faltantes por columna ===")
print(porc_nulos.sort_values(ascending=False).round(2))

# --- Seleccionar columnas a eliminar (más del 5% de nulos) ---
cols_drop = porc_nulos[porc_nulos > 5].index.tolist()

print(f"\nColumnas a eliminar (>5% nulos): {cols_drop}")

# --- Eliminar columnas ---
df_limpio = df.drop(columns=cols_drop)

print(f"\nShape original: {df.shape}")
print(f"Shape después de limpieza: {df_limpio.shape}")

=== Porcentaje de valores faltantes por columna ===
caused_by                  99.98
vendor                     99.83
cmdb_ci                    99.69
rfc                        99.30
problem_id                 98.38
sys_created_by             37.45
sys_created_at             37.45
u_symptom                  23.26
assigned_to                19.40
assignment_group           10.03
opened_by                   3.41
resolved_at                 2.22
closed_code                 0.50
resolved_by                 0.16
subcategory                 0.08
category                    0.06
location                    0.05
caller_id                   0.02
incident_state              0.00
number                      0.00
reassignment_count          0.00
active                      0.00
contact_type                0.00
sys_updated_by              0.00
made_sla                    0.00
sys_mod_count               0.00
reopen_count                0.00
opened_at                   0.00
sys_updated_at          

In [49]:
# --- Convertir a datetime ---
df_limpio["opened_at"] = pd.to_datetime(df_limpio["opened_at"], errors="coerce")
df_limpio["resolved_at"] = pd.to_datetime(df_limpio["resolved_at"], errors="coerce")
df_limpio["closed_at"] = pd.to_datetime(df_limpio["closed_at"], errors="coerce")

# --- Crear métricas derivadas en días ---
df_limpio["resolution_time"] = (df_limpio["resolved_at"] - df_limpio["opened_at"]).dt.total_seconds() / 86400
df_limpio["closure_time"] = (df_limpio["closed_at"] - df_limpio["opened_at"]).dt.total_seconds() / 86400

# --- Estadísticas descriptivas ---
print("\n=== Estadísticas descriptivas de resolution_time ===")
print(df_limpio["resolution_time"].describe().round(2))

print("\n=== Estadísticas descriptivas de closure_time ===")
print(df_limpio["closure_time"].describe().round(2))



=== Estadísticas descriptivas de resolution_time ===
count    138571.00
mean         11.23
std          27.12
min           0.00
25%           0.17
50%           3.06
75%          10.92
max         336.26
Name: resolution_time, dtype: float64

=== Estadísticas descriptivas de closure_time ===
count    56316.00
mean        73.38
std        105.24
min       -289.96
25%        -22.65
50%         67.05
75%        162.27
max        555.63
Name: closure_time, dtype: float64


C:\Users\santy\AppData\Local\Temp\ipykernel_41548\4098534172.py:2: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_limpio["opened_at"] = pd.to_datetime(df_limpio["opened_at"], errors="coerce")
C:\Users\santy\AppData\Local\Temp\ipykernel_41548\4098534172.py:3: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_limpio["resolved_at"] = pd.to_datetime(df_limpio["resolved_at"], errors="coerce")


In [50]:
# Normalizar impact, urgency y priority a solo texto limpio
for col in ["impact", "urgency", "priority"]:
    if col in df_limpio.columns:
        # Convertir a string, quitar números + guiones, recortar espacios
        df_limpio[col] = (
            df_limpio[col]
            .astype(str)
            .str.replace(r"^\d+\s*-\s*", "", regex=True)  # quita "1 -", "2 -" etc.
            .str.strip()  # quita espacios extras
        )

# Ver valores únicos después de limpiar
for col in ["impact", "urgency", "priority"]:
    if col in df_limpio.columns:
        print(f"{col}: {df_limpio[col].unique()}")


impact: ['Medium' 'High' 'Low']
urgency: ['Medium' 'Low' 'High']
priority: ['Moderate' 'High' 'Low' 'Critical']


In [51]:
# --- Asegurar tipo numérico ---
df_limpio["reassignment_count"] = pd.to_numeric(df_limpio["reassignment_count"], errors="coerce")
df_limpio["reopen_count"] = pd.to_numeric(df_limpio["reopen_count"], errors="coerce")

# --- 1) Eliminar registros con reassignment_count > 15 ---
n_eliminados = (df_limpio["reassignment_count"] > 15).sum()
df_limpio = df_limpio[df_limpio["reassignment_count"] <= 15]

# --- 2) Outliers en reassignment_count (criterio IQR) ---
q1_r = df_limpio["reassignment_count"].quantile(0.25)
q3_r = df_limpio["reassignment_count"].quantile(0.75)
iqr_r = q3_r - q1_r
limite_sup_r = q3_r + 1.5 * iqr_r

df_limpio["is_outlier_reassignment"] = df_limpio["reassignment_count"] > limite_sup_r

# --- 3) Outliers en reopen_count (criterio IQR, sin eliminar registros) ---
q1_o = df_limpio["reopen_count"].quantile(0.25)
q3_o = df_limpio["reopen_count"].quantile(0.75)
iqr_o = q3_o - q1_o
limite_sup_o = q3_o + 1.5 * iqr_o

df_limpio["is_outlier_reopen"] = df_limpio["reopen_count"] > limite_sup_o

# --- 4) Asegurar tipo booleano ---
df_limpio["is_outlier_reassignment"] = df_limpio["is_outlier_reassignment"].astype(bool)
df_limpio["is_outlier_reopen"] = df_limpio["is_outlier_reopen"].astype(bool)

# --- 5) Resumen ---
total_post = len(df_limpio)
outliers_reassignment = df_limpio["is_outlier_reassignment"].sum()
outliers_reopen = df_limpio["is_outlier_reopen"].sum()

print("=== Resumen de outliers y limpieza ===")
print(f"Registros eliminados en reassignment_count (>15): {n_eliminados}")
print(f"Outliers en reassignment_count (IQR): {outliers_reassignment} ({outliers_reassignment/total_post*100:.2f}%)")
print(f"Outliers en reopen_count (IQR): {outliers_reopen} ({outliers_reopen/total_post*100:.2f}%)")


=== Resumen de outliers y limpieza ===
Registros eliminados en reassignment_count (>15): 87
Outliers en reassignment_count (IQR): 19548 (13.80%)
Outliers en reopen_count (IQR): 2256 (1.59%)


In [52]:
# --- 1) Filtro temporal: opened_at <= 2017-02-28 ---
cutoff = pd.Timestamp("2017-01-01")
n0 = len(df_limpio)
df_limpio = df_limpio[df_limpio["opened_at"] <= cutoff].copy()
elim_temporal = n0 - len(df_limpio)

# --- 2) Eliminar incoherencias: resolved_at < opened_at o closed_at < opened_at ---
incoh_mask = (
    (df_limpio["resolved_at"].notna() & df_limpio["opened_at"].notna() & (df_limpio["resolved_at"] < df_limpio["opened_at"])) |
    (df_limpio["closed_at"].notna()   & df_limpio["opened_at"].notna() & (df_limpio["closed_at"]   < df_limpio["opened_at"]))
)
elim_incoh = int(incoh_mask.sum())
df_limpio = df_limpio[~incoh_mask].copy()

# --- 3) Métricas derivadas (días) ---
df_limpio["resolution_time"] = (df_limpio["resolved_at"] - df_limpio["opened_at"]).dt.total_seconds() / 86400
df_limpio["closure_time"]    = (df_limpio["closed_at"]   - df_limpio["opened_at"]).dt.total_seconds() / 86400

# --- 4) Flags de calidad ---
# Inválidos: tiempos negativos (no se eliminan, solo se marcan)
df_limpio["is_invalid_time"] = (
    (df_limpio["resolution_time"] < 0) | (df_limpio["closure_time"] < 0)
).fillna(False).astype(bool)

# Outliers: resolution_time > 365 días (no se eliminan, solo se marcan)
df_limpio["is_outlier_resolution_time"] = (df_limpio["resolution_time"] > 365).fillna(False).astype(bool)

# --- 5) Resumen solicitado ---
n_invalid  = int(df_limpio["is_invalid_time"].sum())
n_outliers = int(df_limpio["is_outlier_resolution_time"].sum())

print("=== Resumen limpieza temporal y calidad ===")
print(f"Registros eliminados por filtro temporal (opened_at > 2017-01-01): {elim_temporal}")
print(f"Registros eliminados por inconsistencias (resolved_at/closed_at < opened_at): {elim_incoh}")
print(f"Registros marcados como inválidos (resolution_time < 0 o closure_time < 0): {n_invalid}")
print(f"Registros marcados como outliers (resolution_time > 365 días): {n_outliers}")

=== Resumen limpieza temporal y calidad ===
Registros eliminados por filtro temporal (opened_at > 2017-01-01): 829
Registros eliminados por inconsistencias (resolved_at/closed_at < opened_at): 17602
Registros marcados como inválidos (resolution_time < 0 o closure_time < 0): 0
Registros marcados como outliers (resolution_time > 365 días): 0


In [53]:
# --- Agrupar contact_type ---
df_limpio["contact_type"] = np.where(
    df_limpio["contact_type"] == "Phone", "Phone", "Otros"
)

# --- Verificar resultado ---
print("Valores únicos después de agrupar:")
print(df_limpio["contact_type"].value_counts())

Valores únicos después de agrupar:
contact_type
Phone    122618
Otros       576
Name: count, dtype: int64


In [54]:
###### Priority no es independiente.

# En la tabla que mostraste, cada combinación de impact + urgency siempre da una única priority.
#Ejemplo: Impact = High + Urgency = High → Priority = Critical (100% de los casos).
#Eso significa que priority es una función determinística de impact y urgency, no un dato nuevo.

# --- Eliminar la columna priority ---
df_limpio = df_limpio.drop(columns=["priority"])

# --- Verificar ---
print("Columnas actuales en df_limpio:")
print(df_limpio.columns.tolist())


Columnas actuales en df_limpio:
['number', 'incident_state', 'active', 'reassignment_count', 'reopen_count', 'sys_mod_count', 'made_sla', 'caller_id', 'opened_by', 'opened_at', 'sys_updated_by', 'sys_updated_at', 'contact_type', 'location', 'category', 'subcategory', 'impact', 'urgency', 'knowledge', 'u_priority_confirmation', 'notify', 'closed_code', 'resolved_by', 'resolved_at', 'closed_at', 'resolution_time', 'closure_time', 'is_outlier_reassignment', 'is_outlier_reopen', 'is_invalid_time', 'is_outlier_resolution_time']


In [55]:
# --- Asegurar que las columnas sean numéricas ---
df_limpio["resolution_time"] = pd.to_numeric(df_limpio["resolution_time"], errors="coerce")
df_limpio["closure_time"]    = pd.to_numeric(df_limpio["closure_time"], errors="coerce")

# --- 1) Crear columnas booleanas de outliers ---
df_limpio["is_outlier_resolution_time"] = df_limpio["resolution_time"] > 130
df_limpio["is_outlier_closure_time"]    = df_limpio["closure_time"] > 130

# --- 2) Eliminar registros con resolution_time > 130 ---
n_prev = len(df_limpio)
df_limpio = df_limpio[df_limpio["resolution_time"] <= 130].copy()
n_drop = n_prev - len(df_limpio)

# --- 3) Resumen ---
print("=== Limpieza de tiempos ===")
print(f"Filas eliminadas por resolution_time > 130 días: {n_drop}")
print(f"Outliers closure_time > 130 días (marcados, no eliminados): {df_limpio['is_outlier_closure_time'].sum()}")

# --- 4) Verificación de tipos ---
print("\nTipos de las columnas de flags:")
print(df_limpio[["is_outlier_resolution_time", "is_outlier_closure_time"]].dtypes)

=== Limpieza de tiempos ===
Filas eliminadas por resolution_time > 130 días: 4249
Outliers closure_time > 130 días (marcados, no eliminados): 18851

Tipos de las columnas de flags:
is_outlier_resolution_time    bool
is_outlier_closure_time       bool
dtype: object
